# Dimensionality Reduction & Clustering

**Dimensionality reduction** is an incredibly useful tool in data science as well as neuroscience. Here, we'll explore one main method of dimensionality reduction: **Principal Components Analysis**. We'll first generate a dataset, then perform PCA step-by-step, and then learn the very simple commands to do this in Python. Lastly, we'll use *k*-means clustering to find clusters in the two-dimensional PCA data.


### By the end of this notebook, you'll be able to:
* Run PCA on high dimensional data, including electrophysiology time series
* Use *k*-means clustering to find clusters in your data
* Plot the results of your dimensionality reduction and clustering, coloring each point by either a known feature or by a cluster assignment

## Part 1: Create a dataset of 100 neurons with eight different features.

Here, we will simulate a dataset with eight different features. First, we'll decide on the 'prototype' of six different cells, for eight different features: response latency, somatic volume, cortical depth, maximum firing rate, spontaneous firing rate, spike width, axon length, and dendritic arborization area.

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats

# Specify fields to generate data
generatorFields=['type_num', 'transmission', 'latMean','latStd','volMean','volStd', 'depthMean', \
                 'depthStd','maxrateMean', 'maxrateStd','spontMean','spontStd','widthMean','widthStd', \
                 'axonMean','axonStd','dendriteMean','dendriteStd']

# Specify each property value
Type1=[1, 'excitatory', 14, .5, 150,30,500,20,.9, .1,.02,.01, 1, .05, 160, 20, 180,30 ] #L4 pyramid
Type2=[2, 'excitatory', 15, .5, 120,30,300,20,.8, .1,.07,.01, 1, .04, 150, 20, 150,30 ] #L2 pyramid
Type3=[3, 'inhibitory', 15, 1, 120,30, 300,20, .95,.1,.2,.1, .2,.001,150, 10, 150,10 ] #L2 inhibitory PV
Type4=[4, 'inhibitory', 17, 4, 110,30,300,20, .3, .1,.02, .01,.3,.005,150, 10, 150,40 ] #L2 inhibitory SOM
Type5=[5, 'excitatory', 22, 5, 180,20,800,100,.35,.2,.35, .1,.5,.1, 1000, 500, 200,60 ] #L6 excitatory pyramid
Type6=[6, 'inhibitory', 13, .5, 100,30,500,20,.95,.1,.2,.1, .2,.001,150, 10, 150,10 ] #L4 inhibitory FS

dftype = pd.DataFrame([Type1,Type2,Type3,Type4,Type5,Type6],columns=generatorFields)
dftype

Now, we'll create hundred neurons that are randomly selected from these six different cell types.

In [ ]:
cellFields = ['latency','volume','depth','maxrate','spont','width','axon','dendrite','transmission']
dataset = pd.DataFrame(columns = cellFields) # Inititialize our dataset

for i in range(100): # For one hundred neurons
    tt = np.random.randint(6) # Randomly choose a cell type
    trans = dftype.loc[tt,'transmission'] # Excitatory or inhibitory?
    latency = dftype.loc[tt,'latMean']+np.random.randn()*dftype.loc[tt]['latStd'] # Mean latency, with some jitter
    vol=dftype.loc[tt,'volMean']+np.random.randn()*dftype.loc[tt]['volStd'] # Mean volume, with some jitter
    z = dftype.loc[tt,'depthMean']+np.random.randn()*dftype.loc[tt]['depthStd'] # Mean depth, with some jitter
    maxrate = dftype.loc[tt,'maxrateMean']+np.random.randn()*dftype.loc[tt]['maxrateStd']
    spont = dftype.loc[tt,'spontMean']+np.random.randn()*dftype.loc[tt]['spontStd'] 
    waveWidth = dftype.loc[tt,'widthMean']+np.random.randn()*dftype.loc[tt]['widthStd']
    axon = dftype.loc[tt,'axonMean']+np.random.randn()*dftype.loc[tt]['axonStd'] 
    dendrite = dftype.loc[tt,'dendriteMean']+np.random.randn()*dftype.loc[tt]['dendriteStd']
    
    # Append each simulated property to the dataset
    dataset = dataset.append(pd.DataFrame
    ([[latency,vol,z,maxrate,spont,waveWidth,axon,dendrite,trans]],columns=cellFields),ignore_index=True)
    
print(dataset.shape)
nrows,ncolumns = dataset.shape
dataset.head()

Let's take a look at these eight different factors, and how they compare.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns # This is another plotting package, built really nicely for plotting these types of analyses!

sns.pairplot(dataset, height=1.5)
plt.show()

Clearly, there's a lot going on here. We'll want to use dimensionality reduction to try to understand if we can reduce the understanding of this dataset into a couple of different factors.

For now, we'll drop "transmission" from our dataframe -- we don't want to run PCA on it. Then, we'll normalize each column of data. Normalization isn't strictly *necessary* but it is a really useful for a dataset like this one where different variables have wildly different values (e.g., 0.5 to 300).

In [ ]:
x_data = dataset.drop('transmission',axis=1)
x_data = (x_data - x_data.mean())/x_data.std()
# x_data = pd.DataFrame(stats.zscore(x_data)) # An alternative way to normalize
x_data.head()

## Part 2. Perform PCA, manually.

### 1. Compute the covariance matrix.

Here, we'll calculate the **covariance** between the columns of our dataset. Similar to correlation, a positive covariance means these columns vary together, whereas a negative covariance means these columns vary in opposite directions.

For a more detailed direction of covariance and how it is computed, see [this video](https://youtube.com/watch?v=jcTq2PgU5Vw) and [this video](https://www.youtube.com/watch?v=g-Hb26agBFg). 

In [ ]:
covDATA = x_data.cov()

plt.imshow(covDATA)
plt.colorbar()
plt.title('Covariance Matrix (\u03A3)')
plt.show()

### 2. Calculate the eigenvectors and extract the factors by rotation.
Determine the eigenvectors and eigenvalues of our covariance matrix. This **eigenvector decomposition** tells us the dimensions that are most useful for understanding our data.

The **eigenvectors** tell us the direction we need to stretch the data, the **eigenvalues** tell us *how much* we are stretching the data. We'll use a useful tool from the [numpy linear algebra package](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.eig.html) to compute our eigenvectors and eigenvalues.

For a more detailed explaination of eigenvector decomposition, see [this video](https://www.youtube.com/watch?v=PFDu9oVAE-g). 

In [ ]:
# Use the linalg.eigh to compute the eigenvectors and eigenvalues
eigenvectors, eigenvalues = np.linalg.eigh(covDATA)

plt.imshow(eigenvalues)
plt.colorbar()
plt.show()

In [ ]:
# Check that the sum of our eigenvectors is 8 (the number of variables)
eigenSum = np.sum(eigenvectors)
print(eigenSum)

### 3. Determine the number of factors.
PCA will extract as many factors as there are variables, but we'd like to know how many of these factors are useful in explaining our data. 

There are a few different ways to do this (e.g., a Kaiser criterion), but here we'll use a "[Scree plot](https://en.wikipedia.org/wiki/Scree_plot)", and look for an "elbow" (very scientific, I know). Factors to the left of the elbow are considered meaningful, whereas factors to the right are considered noise.

We will first create the pca model, and then use the `pca.fit_transform()` method to both fit the model and project our data back into the extracted components.

In [ ]:
# Import the PCA package
from sklearn.decomposition import PCA

pca = PCA() # Create a PCA model
x_data_pca = pca.fit_transform(x_data)

Below, we'll plot our results.

In [ ]:
plt.plot(pca.explained_variance_ratio_*8) # Plot the expected variance as a function of the PC
plt.axhline(1,c='k',ls='--') # Plot a horizontal line at 1, the Kaiser criterion

plt.title('Scree plot for 100 simulated neurons')
plt.ylabel('Eigenvalue') 
plt.xlabel('Principal Component #')
plt.show()

Another way of looking at this is by plotting the cumulative variance explained by our principle components:

In [ ]:
# Plot a cumulative sum of the explained variance
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.axhline(.9,c='k',ls='--')
plt.title('Scree plot of Cumulative Variance Explained')
plt.ylabel('Cumulative variance explained')
plt.xlabel('Principle Component #')
plt.show()

As you can see, we would need five principle components to account for more than 90% of the variance in the dataset (which would be a lot).

In [ ]:
# Another thing we can do is look at the explained variance for each PCA.
ex_variance=np.var(x_data_pca,axis=0)
ex_variance_ratio = ex_variance/np.sum(ex_variance)
print(ex_variance_ratio)

### 3. Interpreting the meaning of factors.
This part is really tricky for neural data -- what counts as a *meaningful* factor for brain activity? This part is more or less easy depending on your data and how much you already know about it. 

For spike sorting, these factors are typically something obvious like spike amplitude or spike width, but for behavioral measures or population dynamics, the meaning of factors could be less obvious. Below, we can see how much each feature contributes to the first PCs:

In [ ]:
plt.figure(figsize=(10,5))
plt.imshow(pca.components_,cmap='viridis',)
plt.yticks([0,1,2],['PC1','PC2','PC3'],fontsize=10)
plt.colorbar(orientation='horizontal')
plt.tight_layout()
plt.xticks(range(len(x_data.columns)),x_data.columns,rotation=65)

plt.show()

### 4. Determining the factor values of the original variables

Here, we'll plot the first two dimensions of our PCA.

In [ ]:
plt.figure()
plt.scatter(x_data_pca[:, 0], x_data_pca[:, 1])
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.axis('equal')
plt.show()

## Step 3. Perform PCA, the straightforward way.
Thankfully, we don't need to go through *all* of those steps whenever we want to perform PCA in Python. Really, it comes down to three steps:

In [ ]:
from sklearn.decomposition import PCA  # 1. Choose the model class (PCA) and import that package
pca = PCA(n_components = 2)            # 2. Instantiate the model with hyperparameters
X_2D = pca.fit_transform(x_data)       # 3. Fit the PCA model to our data and transform to two dimensions

Below, we'll plot our data using a useful function from seaborn, [lmplot](https://seaborn.pydata.org/generated/seaborn.lmplot.html).

In [ ]:
dataset['PCA1'] = X_2D[:, 0]
dataset['PCA2'] = X_2D[:, 1]
sns.lmplot("PCA1", "PCA2", hue='transmission', data=dataset, fit_reg=False)
plt.show()

Remember the PCA didn't have any information on whether our cells were excitatory or inhibitory, but clearly it's picking up on features that also divide those cell types. And, it looks like excitatory and inhibitory cells actually might fall into three classes. Let's see what happens if we use *k*-means clustering on our data.

In [ ]:
from sklearn.cluster import KMeans #Import the KMeans model

kmeans = KMeans(n_clusters=2) # Set up a kmeans model with 3 clusters

kmeans.fit(X_2D) # Fit our two dimensional data
y_kmeans = kmeans.predict(X_2D) 

Again, we'll use the `sns.lmplot()` method to nicely plot our data. Here, we'll color each point by its assigned *k*-means cluster.

In [ ]:
# Add our kmeans group to the data for plotting
dataset['kmeans_group'] = y_kmeans

sns.lmplot("PCA1", "PCA2", hue='kmeans_group', data=dataset, fit_reg=False)
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5)
plt.show()

If we have tell *k*-means to use three clusters, it divides it up into three clusters. What happens if we tell it there are two clusters?

### Sidenote: What if we wanted to add another column to our dataframe, with a category we'd try to cluster with?

Let's say we want to create a new dataframe with a categorical column for the depth of cells to our dataframe. We'd use the syntax `df['new column name'] = pd.Series(index=dataset.index,dtype='int64')` to add an empty column with the same index and a specified datatype. We can then put 'shallow' or 'deep' in this column using the following syntax: `df.loc[condition, 'new column name'] = 'value'`:

In [ ]:
dataset['depthCategory'] =pd.Series(index=dataset.index,dtype='int64')
dataset.loc[dataset['depth'] < 500, 'depthCategory'] = 'shallow'
dataset.loc[dataset['depth'] >= 500, 'depthCategory'] = 'deep'
dataset.head()

## Step 4. Perform PCA on real electrophysiology data
We can also use PCA on time series data. Here, we'll work with a sample data set (data_for_exercises.mat) which you can download from [here](https://github.com/marius10p/NeuralDataScienceCSHL2019/blob/master/ByronYuExercises/data_for_exercises.mat). This dataset contains spike counts for 97 neurons recorded using an electrode array over the dorsal premotor cortex of a monkey during a reaching task. Each row of the dataset (728) is a different trial for a different angle of reaching (8 angles total). Trials 1-91 are for reach angle 1, 92-182 for reach angle 2, etc.

You can read more about the experiments that created this dataset [here](https://www.jneurosci.org/content/27/40/10742).

In [ ]:
import scipy.io
from collections import defaultdict
import scipy.stats as sc

numChannels = 97
matIn = scipy.io.loadmat('data_for_exercises.mat')
spike_data = pd.DataFrame(matIn['Xplan'])

<div class="alert alert-success">

**Task**: Take a look at the raw electrophysiology data here by setting up a plot and looping through each row of `spike_data` to add it to the plot. Add a variable `y` which increases each time you run through a loop, allowing you to stack all of these traces (instead of plotting them on top of each other).

</div>

In [ ]:
# Plot the raw data here

fig,ax = plt.subplots(figsize=(15,10))
y = 0

for neuron in spike_data:
    plt.plot(spike_data[neuron]+y)
    y = y+10 # 10 is arbitrary! This is just a way of spacing out our traces.
    
plt.show()

In [ ]:
import numpy as np
np.ndarray(spike_data)

<div class="alert alert-success">

**Task**: Compute the PCA for these data and plot the results (with either `plt.scatter()` or `sns.lmplot()`. For this plot, you do not need a `color` or `hue` argument.

</div>

In [ ]:
# Compute your PCA here

<div class="alert alert-success">

**Task**: As it notes above, *Each row of the dataset (728) is a different trial for a different angle of reaching (8 angles total). Trials 1-91 are for reach angle 1, 92-182 for reach angle 2, etc.* In the previous example of cell features, we colored our final PCA plot by whether the cell was excitatory or inhibitory. Here, we want to color each point on our PCA plot by the reach angle.

To do so, create a column in the spike_data frame called "angles", which contains a list of angle names (can simply be 1, 2, 3, etc. Then, go back to your plot and add this as the `hue` argument.

</div>

In [ ]:
# Assign the angles column here

# About this notebook

The examples derived here are largely from Pascal Wallisch's *Neural Data Science* (Chapter 8), and Jake VanderPlaas's *Neural Data Science Handbook*. Inspiration for the exercises in Step 4 are from [this notebook](https://github.com/marius10p/NeuralDataScienceCSHL2019/blob/master/ByronYuExercises/exercises.pdf) from Byron Yu, featuring data from Krisha Shenoy.